In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import missingno as msno
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
from scipy import stats
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize

from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import collections
import re
import emoji

In [2]:
from preprocessing import *
from statistics import *
from list_to_corpus import *
from url_handler import *
from emoji_handler import *
from helper import *

In [3]:
list_corpus

<function list_to_corpus.list_corpus(lst)>

In [4]:
data=Preprocessing('../data/train.csv', '../data/test.csv')

In [5]:
train_df_1, train_df_0, train_df, test_df = data.train_df_dis, data.train_df_nodis, data.train_df, data.test_df

In [7]:
test_df_keyword=test_df['keyword'].fillna('uns_keyword')
test_df['modified_keyword'] = test_df_keyword
test_df_loc=test_df['location'].fillna('uns_location')
test_df['modified_location'] = test_df_loc
test_df =test_df.drop(['location', 'keyword'], axis=1)

In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 3263 non-null   int64 
 1   text               3263 non-null   object
 2   modified_keyword   3263 non-null   object
 3   modified_location  3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [96]:
import collections
# collections.Counter(train_df[train_df["target"] == 1]["text"].values[1])

In [10]:
len(collections.Counter(test_df.modified_keyword))

222

In [13]:
%%time
corpus = list_corpus(test_df['text'])

CPU times: user 14.4 s, sys: 426 ms, total: 14.8 s
Wall time: 19.6 s


In [14]:
word = [doc.split() for doc in corpus]

In [15]:
lst = []
for item in word:
    for i in item:
        lst.append(i)

In [16]:
len(collections.Counter(lst))

13623

In [17]:
collections.Counter(lst).most_common(10)

[('http', 3906),
 ('...', 344),
 ('like', 323),
 ('amp', 284),
 ("'s", 276),
 ('fire', 273),
 ('get', 248),
 ("n't", 232),
 ('via', 210),
 ('https', 206)]

In [145]:
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(corpus)
X = (X.todense())
X.shape

(7613, 10000)

In [146]:
X

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.35669906, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [18]:
test_df['text'].values[100]

'BROOO HE JUST GOT ANNIHILATED https://t.co/UR7QkqG1wf'

In [200]:
# print(vectorizer.get_feature_names())

In [19]:
# def urls(sent):
#     return re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',sent)
# def url_counts(sent):
#     return len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',sent))
def remove_urls(sent):
    return re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',sent)

In [20]:
s ='Working on Nlp. So much fun  - https://www.helloworld.com, http://www.worldhello.com'
# print(urls(s))
# print(url_counts(s))
print(remove_urls(train_df['text'].values[100]))

.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion 


In [21]:
train_df['text'].values[100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [22]:
s ='Working on Nlp. So much fun  - https://www.helloworld.com, https://www.worldhello.com'
print(urls(s))
print(url_counts(s))
print(remove_urls(remove_urls(s)))

['https://www.helloworld.com, https://www.worldhello.com']
1
Working on Nlp. So much fun  -  


In [23]:
test_df['url_count'] = test_df.text.apply(url_counts)
test_df['urls'] = test_df.text.apply(urls)

In [24]:
test_df['text'] = pd.Series([remove_urls(str_) for str_ in train_df.text])

In [25]:
test_df['urls']=test_df['urls'].apply(list_element)

In [26]:
test_df['urls'] = test_df['urls'].fillna('uns_url')

In [27]:
test_df.head()

,id,text,modified_keyword,modified_location,url_count,urls
0,0,Our Deeds are the Reason of this #earthquake M...,uns_keyword,uns_location,0,uns_url
1,2,Forest fire near La Ronge Sask. Canada,uns_keyword,uns_location,0,uns_url
2,3,All residents asked to 'shelter in place' are ...,uns_keyword,uns_location,0,uns_url
3,9,"13,000 people receive #wildfires evacuation or...",uns_keyword,uns_location,0,uns_url
4,11,Just got sent this photo from Ruby #Alaska as ...,uns_keyword,uns_location,0,uns_url


In [28]:
# train_df = train_df.drop(['Char_length', 'word_count'], axis=1)
test_df.head()

,id,text,modified_keyword,modified_location,url_count,urls
0,0,Our Deeds are the Reason of this #earthquake M...,uns_keyword,uns_location,0,uns_url
1,2,Forest fire near La Ronge Sask. Canada,uns_keyword,uns_location,0,uns_url
2,3,All residents asked to 'shelter in place' are ...,uns_keyword,uns_location,0,uns_url
3,9,"13,000 people receive #wildfires evacuation or...",uns_keyword,uns_location,0,uns_url
4,11,Just got sent this photo from Ruby #Alaska as ...,uns_keyword,uns_location,0,uns_url


In [29]:
s = "Working on Nlp. So much fun 😀"
print("emoji_text : ", *emoji_extraction(s),"\nCount of emojis : ",emoji_count(s))

print("Placing text in place of emojis : ->   ",emoji_to_text(s))

emoji_text :  grinning_face 
Count of emojis :  1
Placing text in place of emojis : ->    Working on Nlp. So much fun grinning_face


In [30]:
%%time
test_df['emoji_count'] = test_df.text.apply(emoji_count)
train_df['emojis'] = test_df.text.apply(emoji_extraction)

CPU times: user 4.73 s, sys: 127 ms, total: 4.86 s
Wall time: 6.65 s


In [31]:
train_df

,id,keyword,location,text,target,emojis
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,[]
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,[]
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,[]
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,[]
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,[]
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,NaN
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,NaN
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,NaN
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,NaN


In [32]:
%%time
test_df['text'] = test_df['text'].apply(emoji_to_text)

CPU times: user 2.28 s, sys: 68.3 ms, total: 2.35 s
Wall time: 3.57 s


In [33]:
test_df['text'] = test_df.text.apply(remove_urls)

In [36]:
test_df.head()

,id,text,modified_keyword,modified_location,url_count,urls,emoji_count
0,0,Our Deeds are the Reason of this #earthquake M...,uns_keyword,uns_location,0,uns_url,0
1,2,Forest fire near La Ronge Sask. Canada,uns_keyword,uns_location,0,uns_url,0
2,3,All residents asked to 'shelter in place' are ...,uns_keyword,uns_location,0,uns_url,0
3,9,"13,000 people receive #wildfires evacuation or...",uns_keyword,uns_location,0,uns_url,0
4,11,Just got sent this photo from Ruby #Alaska as ...,uns_keyword,uns_location,0,uns_url,0


In [37]:
corpus = list_corpus(test_df['text'])
word = [doc.split() for doc in corpus]
lst = []
for item in word:
    for i in item:
        lst.append(i)

In [38]:
len(collections.Counter(lst))

13166

In [40]:
collections.Counter(lst).most_common(100)

[("'s", 326),
 ('like', 320),
 ('amp', 302),
 ('...', 284),
 ("n't", 266),
 ('get', 251),
 ('via', 216),
 ("'m", 208),
 ('fire', 196),
 ('new', 184),
 ('one', 183),
 ('bomb', 182),
 ('crash', 170),
 ('burn', 166),
 ('2', 166),
 ('bag', 149),
 ('derail', 144),
 ('time', 141),
 ('train', 141),
 ('attack', 138),
 ('emerg', 136),
 ('two', 135),
 ('build', 134),
 ('still', 132),
 ('emergency', 130),
 ('drown', 130),
 ('go', 128),
 ('death', 127),
 ('man', 126),
 ('know', 126),
 ('video', 119),
 ('got', 116),
 ('destroy', 116),
 ('would', 114),
 ('back', 114),
 ('news', 114),
 ('bodi', 113),
 ("'re", 112),
 ('body', 112),
 ('burning', 112),
 ('us', 110),
 ('day', 110),
 ('car', 109),
 ('collaps', 107),
 ('buildings', 106),
 ('crush', 105),
 ('hiroshima', 104),
 ('ca', 102),
 ('today', 102),
 ('love', 101),
 ('dead', 100),
 ('see', 99),
 ('year', 96),
 ('say', 96),
 ('full', 94),
 ('look', 93),
 ('want', 93),
 ('cross', 93),
 ('peopl', 91),
 ('plan', 91),
 ('people', 90),
 ('rt', 89),
 ('biot

In [44]:
test_df['text'].values

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'Fully Engulfed Garage Fire: Propane Tanks Inside. Sunnymeade Dr.',
       'He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam',
       '@suelinflower there is no words to describe the physical painthey ripped you apart while you screamed for dear lifeits like been engulfed'],
      dtype=object)

In [46]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 3263 non-null   int64 
 1   text               3263 non-null   object
 2   modified_keyword   3263 non-null   object
 3   modified_location  3263 non-null   object
 4   url_count          3263 non-null   int64 
 5   urls               3263 non-null   object
 6   emoji_count        3263 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 178.6+ KB


In [49]:
test_df.head(5)

,id,text,modified_keyword,modified_location,url_count,urls,emoji_count
0,0,Our Deeds are the Reason of this #earthquake M...,uns_keyword,uns_location,0,uns_url,0
1,2,Forest fire near La Ronge Sask. Canada,uns_keyword,uns_location,0,uns_url,0
2,3,All residents asked to 'shelter in place' are ...,uns_keyword,uns_location,0,uns_url,0
3,9,"13,000 people receive #wildfires evacuation or...",uns_keyword,uns_location,0,uns_url,0
4,11,Just got sent this photo from Ruby #Alaska as ...,uns_keyword,uns_location,0,uns_url,0


In [50]:
processed_test = test_df.copy()

In [51]:
processed_test.to_csv('../data/processed_test.csv', index=False)

In [52]:
processed_test

,id,text,modified_keyword,modified_location,url_count,urls,emoji_count
0,0,Our Deeds are the Reason of this #earthquake M...,uns_keyword,uns_location,0,uns_url,0
1,2,Forest fire near La Ronge Sask. Canada,uns_keyword,uns_location,0,uns_url,0
2,3,All residents asked to 'shelter in place' are ...,uns_keyword,uns_location,0,uns_url,0
3,9,"13,000 people receive #wildfires evacuation or...",uns_keyword,uns_location,0,uns_url,0
4,11,Just got sent this photo from Ruby #Alaska as ...,uns_keyword,uns_location,0,uns_url,0
...,...,...,...,...,...,...,...
3258,10861,Do you feel engulfed with low self-image? Take...,uns_keyword,uns_location,0,uns_url,0
3259,10865,Man is equally incapable of seeing the nothing...,uns_keyword,uns_location,0,uns_url,0
3260,10868,Fully Engulfed Garage Fire: Propane Tanks Insi...,uns_keyword,uns_location,1,http://t.co/UtbXLcBIuY,0
3261,10874,He came to a land which was engulfed in tribal...,uns_keyword,uns_location,1,http://t.co/3X6RBQJHn3,0
